In [33]:
from db_info import user, password, host, port, database
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
user = user,
password = password,
host = host,
port = port,
database = database,
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_content")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

In [38]:
cursor.execute("select * from injury_list il inner join injury_content ic on il.판례일련번호=ic.판례정보일련번호")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   판례일련번호    4189 non-null   int64 
 1   사건명       4188 non-null   object
 2   사건번호      4189 non-null   object
 3   선고일자      4189 non-null   object
 4   법원명       4189 non-null   object
 5   사건종류명     4189 non-null   object
 6   판례상세링크    4189 non-null   object
 7   판례정보일련번호  4189 non-null   int64 
 8   사건번호      4189 non-null   object
 9   선고        4189 non-null   object
 10  판결유형      4189 non-null   object
 11  판시사항      3417 non-null   object
 12  판결요지      3028 non-null   object
 13  참조조문      3392 non-null   object
 14  참조판례      1925 non-null   object
 15  판례내용      4189 non-null   object
dtypes: int64(2), object(14)
memory usage: 523.8+ KB


In [5]:
import re
import pandas as pd

df_split = df.copy()
# 판례본문 내용 분리 - 이유 기준 분리

# '【이' 앞부분과 '【이'를 포함한 뒷부분으로 나누는 코드
df_split['판례내용_상단'] = df['판례내용'].str.split('【이').str[0]
df_split['판례내용_이유'] = df['판례내용'].str.extract(r'(【이.*)')

# 판례내용 상단에서 재판당사자 추출

# '문단의 첫 번째 '【'부터 '【주' 앞까지의 텍스트를 포함하여 추출
def extract_text_with_first_bracket_to_ju(content):
    # 정규식으로 첫 번째 '【'부터 '【주'까지 텍스트 추출 (첫 번째 '【' 포함)
    match = re.search(r'(【.*?【\s*주)', content, re.DOTALL)
    if match:
        return match.group(1).strip()  # 추출한 텍스트의 양쪽 공백 제거
    return None  # 매치되지 않으면 None 반환

# '재판당사자' 열 생성 (첫 번째 '【'부터 '【주'까지 추출, 첫 번째 '【' 포함)
df_split['재판당사자'] = df_split['판례내용_상단'].apply(extract_text_with_first_bracket_to_ju)

# 결과 확인
#df_service_split.head()

def extract_and_clean_text(content):
    if isinstance(content, str):  # 내용이 문자열인지 확인
        matches = re.findall(r'【(.*?)】', content)  # '【'와 '】' 사이의 모든 텍스트 찾기
        cleaned_texts = [match.replace(' ', '') for match in matches]  # 공백 제거
        return cleaned_texts  # 결과 리스트 반환
    return []  # 문자열이 아닌 경우 빈 리스트 반환

# 결과 열 생성
df_split['재판당사자_텍스트'] = df_split['재판당사자'].apply(extract_and_clean_text)

# 결과 확인
df_split[['재판당사자_텍스트']]

# 판례내용 이유 부분의 대괄호를 삭제하여 본문으로 변환


service_reason = []

for idx in range(len(df_split)):
    content = df_split['판례내용_이유'].iloc[idx]
    
    # 문자열인지 확인 (문자열이 아니면 건너뜀)
    if isinstance(content, str):
        content_parts = re.split(r'【(.*?)】', content)
        reason_dict = {}

        for j in range(len(content_parts)):
            if content_parts[j].replace(' ', '') in ['이유']:
                cleaned_content = content_parts[j+1]
                cleaned_content = cleaned_content.replace('<br/>', '').replace('\r', '').replace('\n', '')
                cleaned_content = cleaned_content.replace('【', '').replace('】', '').strip()
                cleaned_content = re.sub(r'\s+', ' ', cleaned_content)
                reason_dict[content_parts[j].replace(' ', '')] = cleaned_content
        
        service_reason.append(reason_dict)
    else:
        # 비문자열 데이터 처리 (필요한 경우 빈 딕셔너리 또는 다른 기본값)
        service_reason.append({})

# 결과를 '판례내용_이유2' 열에 저장
df_split['판례내용_이유2'] = service_reason

df_split.info()

# 전처리 함수 정의
def preprocess_content(content):
    if pd.isna(content):  # NaN 값에 대한 예외 처리
        return content
    
    # content가 문자열인지 확인
    if isinstance(content, str):
        content = content.replace('<br/>', '')
        content = content.replace('\n', '')
        content = content.replace('\r', '')
    
    return content

# 여러 열에 전처리를 적용할 경우
columns_to_preprocess = ['참조조문','참조판례', '판례내용', '판례내용_상단', '판례내용_이유', '재판당사자']

# applymap이 아닌 각 열에 개별적으로 apply 사용
for col in columns_to_preprocess:
    df_split[col] = df_split[col].apply(preprocess_content)

# 전처리 후 데이터프레임 확인
df_split.head()

df2=df_split
df2.to_csv('data/hurtcombined.csv')

# 판례데이터를 읽은 뒤 목록과 본문을 합친 뒤 분할해서 전처리

NameError: name 'df' is not defined

In [2]:
import pandas as pd
df2=pd.read_csv('data/hurtcombined.csv')

In [6]:
from transformers import BertModel, DistilBertModel, BertTokenizer
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

In [8]:
df2['판시사항'] = df2['판시사항'].fillna('')


In [9]:
from kobert_transformers import get_tokenizer
import torch

model = distilbert_model  # 모델 선언
tokenizer = get_tokenizer()  # 토크나이저 선언

# 각 문장을 토큰화, id화
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in df2['판시사항']]
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

In [10]:
len(tokenid_list)


4189

In [12]:
# sequence를 같은 길이로 padding
max_length = max(len(ids) for ids in tokenid_list)
padded_tokenid_list = [ids + [0] * (max_length - len(ids)) for ids in tokenid_list]

In [13]:
# Longtensor로 dtype변경
tokenid_tensor = torch.LongTensor(padded_tokenid_list)


In [14]:
# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

RuntimeError: The size of tensor a (924) must match the size of tensor b (512) at non-singleton dimension 1

그냥 넣으면 내용이 너무 길어져서 안됐고 OKT 토큰화한 값을 넣었으나 이 또한 마찬가지로 메모리 부족이 발생한다.

In [15]:
def sliding_window_tokenize(tokens, window_size=256, stride=128):
    """
    긴 토큰 리스트를 슬라이딩 윈도우 방식으로 나누는 함수.
    
    :param tokens: 토큰화된 리스트
    :param window_size: 각 윈도우의 크기
    :param stride: 겹치는 부분의 길이
    :return: 슬라이딩 윈도우로 나눈 토큰 리스트들의 리스트
    """
    token_windows = []
    for start in range(0, len(tokens), stride):
        end = start + window_size
        token_window = tokens[start:end]
        if len(token_window) == window_size:  # 윈도우 크기에 맞는 경우에만 추가
            token_windows.append(token_window)
    return token_windows

# 예제: 긴 토큰화된 문장을 슬라이딩 윈도우 방식으로 나누기
tokenized_sentence = tokenizer.tokenize(df2['판시사항'][0])
token_windows = sliding_window_tokenize(tokenized_sentence, window_size=256, stride=128)

# 각 토큰 윈도우를 확인
for i, window in enumerate(token_windows):
    print(f"윈도우 {i + 1}: {window}")

윈도우 1: ['▁[', '1', ']', '▁국내', '사업', '장을', '▁가진', '▁외국', '법인', '이', '▁국내', '▁및', '▁국', '외', '에', '▁걸쳐', '▁선박', '에', '▁의한', '▁국제', '운', '송', '업', '을', '▁영', '위', '하며', '▁발생', '하는', '▁소득', '▁중', '▁국내에서', '▁승', '선', '한', '▁여', '객', '이나', '▁선', '적', '한', '▁화', '물', '에', '▁관련', '하여', '▁발생한', '▁소득', '만', '이', '▁구', '▁법인', '세', '법', '▁제', '93', '조', '▁제', '5', '호', '에', '▁따른', '▁해당', '▁외국', '법인', '의', '▁국내', '원', '천', '▁사업', '소득', '으로서', '▁법인', '세', '▁과세', '표', '준', '에', '▁포함', '되는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '2', ']', '▁외국', '법인', '의', '▁국내', '원', '천', '소득', '▁총', '합', '계', '액', '에', '▁포함', '되는', '▁‘', '선', '박', '의', '▁외국', '항', '행', '소득', '’', '의', '▁존재', '▁및', '▁범위', '는', '▁원칙', '적으로', '▁과세', '관', '청', '이', '▁증명', '하여', '야', '▁하는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '3', ']', '▁외국', '항', '행', '용', '역', '을', '▁제공하는', '▁외국', '법인', '에', '▁구', '▁부', '가', '가', '치', '세', '법', '령', '상', '▁영', '세', '율이', '▁적용', '되는', '▁과세', '표', '준',

In [16]:
token_ids_list = []
for tokens in token_windows:
    # [CLS]와 [SEP] 토큰 추가
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    # 토큰을 ID로 변환
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    # 패딩 추가 (길이를 max_length로 맞추기)
    token_ids = token_ids + [0] * (max_length - len(token_ids))
    token_ids_list.append(token_ids)

# 텐서로 변환
tokenid_tensor = torch.LongTensor(token_ids_list)

In [17]:
print(tokenid_tensor)

tensor([[   2,  702,   93,  363, 1138, 6503, 7187,  769, 3471, 6338, 7096, 1138,
         2184, 1132, 6995, 6896,  894, 2741, 6896, 3642, 1155, 7010, 6621, 6873,
         7088, 3376, 7044, 7810, 2243, 7794, 2828, 4257, 1139, 2949, 6559, 7828,
         3298, 5370, 7098, 2734, 7202, 7828, 5112, 6241, 6896, 1083, 7815, 2244,
         2828, 6150, 7096, 1115, 2327, 6579, 6335, 4128,  242, 7253, 4128,  172,
         7925, 6896, 1839, 5000, 3471, 6338, 7095, 1138, 7020, 7422, 2609, 6608,
         7080, 2327, 6579, 1060, 7741, 7288, 6896, 4864, 5889, 7318, 3310,   18,
         7202, 5543,   40,  253,  380,  435,   59,  257,  702,  119,  363, 3471,
         6338, 7095, 1138, 7020, 7422, 6608, 4512, 7842, 5436, 6835, 6896, 4864,
         5889,  712, 6559, 6280, 7095, 3471, 7846, 7881, 6608,  498, 7095, 4193,
         2184, 2319, 5760, 3543, 7203, 1060, 5474, 7431, 7096, 4293, 7815, 6844,
         4930, 7318, 3310,   18, 7202, 5543,   40,  253,  380,  435,   59,  257,
          702,  142,  363, 3

In [18]:
from kobert_transformers import get_tokenizer
import torch

def prepare_inputs_for_kobert(token_windows, max_length=256):
    """
    슬라이딩 윈도우로 나눈 토큰들을 KoBERT 입력 형식에 맞게 전처리하는 함수.
    
    :param token_windows: 슬라이딩 윈도우로 나눈 토큰 리스트
    :param max_length: 각 입력의 최대 길이 (윈도우 크기와 동일해야 함)
    :return: 전처리된 토큰 ID 텐서
    """
    token_ids_list = []
    for tokens in token_windows:
        # [CLS]와 [SEP] 토큰 추가
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        # 토큰을 ID로 변환
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        # 패딩 추가 (길이를 max_length로 맞추기)
        token_ids = token_ids + [0] * (max_length - len(token_ids))
        token_ids_list.append(token_ids)
    
    # 텐서로 변환
    tokenid_tensor = torch.LongTensor(token_ids_list)
    return tokenid_tensor

# 긴 문장을 슬라이딩 윈도우 방식으로 나누기
tokenized_sentence = tokenizer.tokenize("여기에 긴 문장을 입력하세요.")
token_windows = sliding_window_tokenize(tokenized_sentence, window_size=256, stride=128)

# 전처리하여 KoBERT에 입력할 텐서 준비
tokenid_tensor = prepare_inputs_for_kobert(token_windows, max_length=256)

# 텐서 출력 확인
print(tokenid_tensor)

tensor([], dtype=torch.int64)


In [22]:
print(df2['판시사항'].isnull().sum())  # NaN 개수 확인
print(df2['판시사항'].head())  # 처음 5개 데이터 확인
print(len(df2['판시사항']))  # 데이터프레임의 길이 확인

0
0     [1] 국내사업장을 가진 외국법인이 국내 및 국외에 걸쳐 선박에 의한 국제운송업을...
1      특수상해죄 및 특수협박죄의 구성요건 중 위험한 물건을 ‘휴대하여’의 의미 / 이...
2    형사소송법 제33조 제1항 제1호에서 필요적 국선변호인 선정사유 중 하나로 정한 ‘...
3      피고인이 별거 상태로 이혼소송 중이던 배우자 甲(女)이 거주하는 원룸에 베란다를...
4                                                     
Name: 판시사항, dtype: object
4189


In [23]:
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in df2['판시사항']]

# 토큰화된 내용 확인
print(tokenized_sentences[:5])  # 처음 5개 문장 출력

[['▁[', '1', ']', '▁국내', '사업', '장을', '▁가진', '▁외국', '법인', '이', '▁국내', '▁및', '▁국', '외', '에', '▁걸쳐', '▁선박', '에', '▁의한', '▁국제', '운', '송', '업', '을', '▁영', '위', '하며', '▁발생', '하는', '▁소득', '▁중', '▁국내에서', '▁승', '선', '한', '▁여', '객', '이나', '▁선', '적', '한', '▁화', '물', '에', '▁관련', '하여', '▁발생한', '▁소득', '만', '이', '▁구', '▁법인', '세', '법', '▁제', '93', '조', '▁제', '5', '호', '에', '▁따른', '▁해당', '▁외국', '법인', '의', '▁국내', '원', '천', '▁사업', '소득', '으로서', '▁법인', '세', '▁과세', '표', '준', '에', '▁포함', '되는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '2', ']', '▁외국', '법인', '의', '▁국내', '원', '천', '소득', '▁총', '합', '계', '액', '에', '▁포함', '되는', '▁‘', '선', '박', '의', '▁외국', '항', '행', '소득', '’', '의', '▁존재', '▁및', '▁범위', '는', '▁원칙', '적으로', '▁과세', '관', '청', '이', '▁증명', '하여', '야', '▁하는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '3', ']', '▁외국', '항', '행', '용', '역', '을', '▁제공하는', '▁외국', '법인', '에', '▁구', '▁부', '가', '가', '치', '세', '법', '령', '상', '▁영', '세', '율이', '▁적용', '되는', '▁과세', '표', '준', '을', 

In [24]:
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

# ID 리스트 길이 확인
print([len(ids) for ids in tokenid_list][:5])  # 처음 5개 문장 길이 확인

# 최대 길이 설정
max_length = 512

# 패딩 처리
padded_tokenid_list = [ids[:max_length] + [0] * (max_length - len(ids[:max_length])) for ids in tokenid_list]

# 패딩된 리스트 길이 확인
print([len(ids) for ids in padded_tokenid_list][:5])  # 처음 5개 문장 길이 확인

[307, 118, 115, 199, 0]
[512, 512, 512, 512, 512]


In [25]:
# LongTensor로 변환
tokenid_tensor = torch.LongTensor(padded_tokenid_list)

# 텐서 크기 확인
print(tokenid_tensor.shape)  # 출력: (batch_size, max_length)

torch.Size([4189, 512])


In [26]:

# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

# 출력은 (batchsize, 토큰id의 길이, 768). 여기서 768은 임베딩 공간의 크기

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 52709818368 bytes.

In [32]:
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in okt_tokens]
print(tokenid_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 